In [1]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227141 sha256=c836171bb6fb86f9a5f6507ce67869ad19895a990780fd2c2d991cb0d027b28a
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [79]:
import fasttext
import pandas as pd

Đọc file csv chứa các URL đã được phân loại và đưa vào data frame

In [80]:
df= pd.read_csv("fasttext.csv")

In [ ]:
dframe = pd.read_csv("fasttext.csv")
dframe

,type,URL
0,safe,https://www.google.com
1,safe,https://www.youtube.com
2,safe,https://www.facebook.com
3,safe,https://www.baidu.com
4,safe,https://www.wikipedia.org
...,...,...
4745,phishing,http://reactivate-device.web.app/
4746,phishing,http://loginorange5427812232335566363636838636...
4747,phishing,http://telegramcn.top/
4748,phishing,https://btinternet-106352.weeblysite.com/


In [24]:
df.head(5)

,type,URL,type_URL
0,safe,https://www.google.com,__label__safe https www google com
1,safe,https://www.youtube.com,__label__safe https www youtube com
2,safe,https://www.facebook.com,__label__safe https www facebook com
3,safe,https://www.baidu.com,__label__safe https www baidu com
4,safe,https://www.wikipedia.org,__label__safe https www wikipedia org


In [25]:
df.type.value_counts()
# df.type.unique()

type
phishing    2603
safe        2147
Name: count, dtype: int64

Hàm tiền xử lý dữ liệu: Các URL trước khi được phân loại sẽ được tách thành các từ riêng biệt và loại bỏ các phần giống nhau (VD: https, www,...)

In [78]:
import re

def preprocess(text):
  text = re.sub(r'^https?://',' ', text).strip()
  text = re.sub(r'^www?\.', ' ', text)
  text = re.sub(r'[^\w\s]', ' ', text)
  text = re.sub(r'[_]', ' ', text)
  text = re.sub(r' +', '', text)
  return text.lower().strip()

In [45]:
preprocess("https://www.familysearch.org/learn/wiki/en/Crawford_County%2C_Pennsylvania")

'familysearch org learn wiki en crawford county 2c pennsylvania'

In [46]:
df['type_URL']=df['type'] + " " + df['URL'].map(preprocess)

In [47]:
df['type_URL'] = "__label__" + df['type_URL'].astype(str)

In [48]:
df['type_URL'][0]

'__label__safe google com'

In [ ]:
# df['type_URL'] = df['type_URL'].map(preprocess)

In [77]:
df['type_URL'][4000]

'__label__phishing app aave com com'

In [61]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
train.shape, test.shape
train.to_csv("URL.train", columns=["type_URL"], index=False, header=False)
test.to_csv("URL.test", columns=["type_URL"], index=False, header=False)
model = fasttext.train_supervised('URL.train', lr = 0.05, ws = 6, epoch = 100)
model.test('URL.test')

(950, 0.8789473684210526, 0.8789473684210526)

In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
train.shape, test.shape

((3800, 3), (950, 3))

In [14]:
train.to_csv("URL.train", columns=["type_URL"], index=False, header=False)
test.to_csv("URL.test", columns=["type_URL"], index=False, header=False)

In [15]:
model = fasttext.train_supervised('URL.train')

In [16]:
model

In [17]:
model.test('URL.test')


(950, 0.9610526315789474, 0.9610526315789474)

In [64]:
with open('validate.txt', 'r') as g:
    urls = [line.rstrip() for line in g.readlines()]
g.close()
wrong = 0
for i, url in enumerate(urls):
  typeURL = "Safe" if "safe" in str(model.predict(preprocess(url))[0]) else "Phishing"
  if "Safe" in typeURL and i <= 13: wrong += 1
  if "Phishing" in typeURL and i > 13: wrong += 1
  print(f"Du doan tai URL thu {i + 1} la: {typeURL}")
print(f"Tong so truong hop du doan sai {wrong}")
print(f"Ti le du doan dung tren {len(urls)} URL: {(len(urls) - wrong) / len(urls) * 100}%")

Du doan tai URL thu 1 la: Safe
Du doan tai URL thu 2 la: Phishing
Du doan tai URL thu 3 la: Phishing
Du doan tai URL thu 4 la: Phishing
Du doan tai URL thu 5 la: Phishing
Du doan tai URL thu 6 la: Phishing
Du doan tai URL thu 7 la: Phishing
Du doan tai URL thu 8 la: Phishing
Du doan tai URL thu 9 la: Phishing
Du doan tai URL thu 10 la: Phishing
Du doan tai URL thu 11 la: Safe
Du doan tai URL thu 12 la: Phishing
Du doan tai URL thu 13 la: Phishing
Du doan tai URL thu 14 la: Safe
Du doan tai URL thu 15 la: Safe
Du doan tai URL thu 16 la: Safe
Du doan tai URL thu 17 la: Safe
Du doan tai URL thu 18 la: Safe
Du doan tai URL thu 19 la: Safe
Du doan tai URL thu 20 la: Safe
Du doan tai URL thu 21 la: Safe
Du doan tai URL thu 22 la: Safe
Du doan tai URL thu 23 la: Safe
Du doan tai URL thu 24 la: Safe
Du doan tai URL thu 25 la: Safe
Du doan tai URL thu 26 la: Safe
Du doan tai URL thu 27 la: Safe
Du doan tai URL thu 28 la: Safe
Tong so truong hop du doan sai 3
Ti le du doan dung tren 28 URL: 89.2

In [70]:
print(model.predict(preprocess("https://orangeespace.godaddysites.com/")))

(('__label__phishing',), array([0.99912554]))


In [42]:
model.get_nearest_neighbors("wallet")

[(0.9956079125404358, '58721b28326e42ec8f18de247ae921dc'),
 (0.9956074953079224, 'ipr'),
 (0.9956074953079224, 'diom'),
 (0.9956073760986328, 'bf3c2f6aa6974504bfea991c6f91264c'),
 (0.9956066608428955, 'wormholeairdrop'),
 (0.9956066608428955, 'trust'),
 (0.9956058859825134, 'allgegro'),
 (0.9956055879592896, 'dubaibornapes'),
 (0.9956055879592896, 'th'),
 (0.99560546875, 'resource')]

In [ ]:
model.save_model("fasttext.model")

Test pretrained model

In [21]:
model = fasttext.load_model("fasttext.model")

In [22]:
print(model.predict(preprocess("https://www.instagram.com/")))

(('__label__phishing',), array([0.88972867]))


In [23]:
with open('validate.txt', 'r') as g:
    urls = [line.rstrip() for line in g.readlines()]
g.close()
wrong = 0
for i, url in enumerate(urls):
  typeURL = "Safe" if "safe" in str(model.predict(preprocess(url))[0]) else "Phishing"
  if "Safe" in typeURL and i <= 13: wrong += 1
  if "Phishing" in typeURL and i > 13: wrong += 1
  print(f"Du doan tai URL thu {i + 1} la: {typeURL}")
print(f"Tong so truong hop du doan sai {wrong}")
print(f"Ti le du doan dung tren {len(urls)} URL: {(len(urls) - wrong) / len(urls) * 100}%")

Du doan tai URL thu 1 la: Phishing
Du doan tai URL thu 2 la: Phishing
Du doan tai URL thu 3 la: Phishing
Du doan tai URL thu 4 la: Phishing
Du doan tai URL thu 5 la: Phishing
Du doan tai URL thu 6 la: Phishing
Du doan tai URL thu 7 la: Phishing
Du doan tai URL thu 8 la: Safe
Du doan tai URL thu 9 la: Phishing
Du doan tai URL thu 10 la: Phishing
Du doan tai URL thu 11 la: Safe
Du doan tai URL thu 12 la: Phishing
Du doan tai URL thu 13 la: Phishing
Du doan tai URL thu 14 la: Phishing
Du doan tai URL thu 15 la: Phishing
Du doan tai URL thu 16 la: Phishing
Du doan tai URL thu 17 la: Safe
Du doan tai URL thu 18 la: Safe
Du doan tai URL thu 19 la: Safe
Du doan tai URL thu 20 la: Phishing
Du doan tai URL thu 21 la: Phishing
Du doan tai URL thu 22 la: Phishing
Du doan tai URL thu 23 la: Phishing
Du doan tai URL thu 24 la: Phishing
Du doan tai URL thu 25 la: Safe
Du doan tai URL thu 26 la: Phishing
Du doan tai URL thu 27 la: Phishing
Du doan tai URL thu 28 la: Safe
Tong so truong hop du doan sa